# Combining Our Local Data Sources

## Combining `.tif` File Data

In [3]:
from rasterio.plot import plotting_extent
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio

In [5]:
# gsa

tif_files = {}
datasets = {}
for i in range (1, 2):
    if i < 10:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_0{i}.tif"
    else:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_{i}.tif"
    tif_files[i] = tif_file_wst
    
    # Reading the .tif file
    with rasterio.open(tif_file_wst) as ds:
        pvo_data = ds.read(1)  # Reading the first band
        
        # Data dimensions
        h = ds.height
        w = ds.width
        t = ds.transform

        # Data values
        lats = []
        longs = []
        pvo_vals = []

        # Extracting the information of each data points from processed .tif file
        for row in range(h):
            for col in range(w):
                pvo_val = pvo_data[row, col]
                if pvo_val != ds.nodata and not np.isnan(pvo_val):
                    
                    # Converting pixel coords to geographic coords
                    lat, long = ds.xy(t, row, col)
                    lats.append(lat)
                    longs.append(long)
                    pvo_vals.append(pvo_val)

        # Creating a DataFrame
        df = pd.DataFrame({
            'latitude': lats,
            'longitude': longs,
            'pvo': pvo_vals
        })
        datasets[i] = df            

print(datasets[1].head())

KeyboardInterrupt: 

In [6]:
# nrel

irradiance_file = "../Data/nrel/monthly-ghi/ghi_apr.tif"

with rasterio.open(irradiance_file) as ds:
    irr_data = ds.read(1) 
    
    height = ds.height
    width = ds.width
    transform = ds.transform
    
    lats, longs, irr_vals = [], [], []
    
    # Extract data points
    for row in range(height):
        for col in range(width):
            irr_val = irr_data[row, col]
            if irr_val != ds.nodata and not np.isnan(irr_val):
                # Convert pixel coords to geographic coords
                long, lat = rasterio.transform.xy(transform, row, col)
                lats.append(lat)
                longs.append(long)
                irr_vals.append(irr_val)
    
    irr_df = pd.DataFrame({
        'latitude': lats,
        'longitude': longs,
        'irradiance': irr_vals
    })

print(irr_df.head())

   latitude  longitude  irradiance
0     59.97    -167.58  138.034978
1     59.97    -167.54  138.196966
2     59.97    -167.50  139.161330
3     59.97    -167.46  140.819042
4     59.97    -167.42  139.861075
